## Prepare training data

In [ ]:
# %%capture cap
# see MTD with QE: https://www.quantum-espresso.org/wp-content/uploads/2022/03/plumed_quick_ref.pdf
# 

%load_ext autoreload
%autoreload 2


import os

os.environ['OMP_NUM_THREADS'] = "6,1"
os.environ["ASE_DFTB_COMMAND"] = "ulimit -s unlimited; /home/dlb/anaconda3/envs/dftb_plumed/bin/dftb+ > PREFIX.out"
os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"

# https://www.plumed.org/doc-v2.7/user-doc/html/_m_e_t_a_d.html
# https://github.com/dftbplus/dftbplus/issues/1064
# try with numpy 1.22　
# https://gitlab.com/Sucerquia/ase-plumed_tutorial

from ase.calculators.lj import LennardJones
# from ase.calculators.plumed import Plumed
from runner.plumed import Plumed
# from plumed import Plumed
from ase.constraints import FixedPlane
from ase.io import read
from ase import units

from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths

# import plumed

from ase.calculators.dftb import Dftb



ps = 1000 * units.fs

setup = [f"UNITS LENGTH=A TIME={1/ps} ENERGY={units.mol/units.kJ}",
        #  "d: DISTANCE ATOMS=1,2 ",
        "at8: FIXEDATOM AT=-0.3652684296711638,5.59639757164289,11.089411052917308",
        "at6: FIXEDATOM AT=-0.5388786945959876,6.471914514609469,10.07028636148657",
        "c1: CENTER ATOMS=at8,at6",
        "c2: CENTER ATOMS=67,69,73,77,79,83",
        "c3: CENTER ATOMS=7,9,11,13,15,17",
        "c4: CENTER ATOMS=79,83",
         "phi: TORSION ATOMS=c1,c2,c3,c4",
         "metad: METAD ARG=phi SIGMA=1.10 HEIGHT=15.0 BIASFACTOR=5 TEMP=300.0 PACE=1", #GRID_MIN=-pi GRID_MAX=pi GRID_BIN=1500
         # "ARG=d SIGMA=0.20,0.20 HEIGHT=1.20 BIASFACTOR=5 TEMP=300.0 PACE=500",
         # "GRID_MIN=0,0 GRID_MAX=1.0,1.0 GRID_BIN=150,150",
         "PRINT ARG=phi STRIDE=1 FILE=../results/test/plumed/COLVAR",
         # "CALC_RCT ",
         # "RCT_USTRIDE=10",
         # "...",
         "FLUSH STRIDE=1000"]


atoms = read('../data/structures/new_systems/ktu_002.cif')


# fix_atoms = FixAtoms(indices=[atom.index for atom in atoms if atom.symbol=='N' or atom.symbol=='Si' or atom.symbol=='O'])
# # # ch_bonds = Analysis(atoms).get_bonds("C", "H")[0]
# # # fix_bond_lengths = FixBondLengths(ch_bonds)
# # # atoms.set_constraint([fix_atoms, fix_bond_lengths])
# atoms.set_constraint(fix_atoms)

calc_base = Dftb(atoms=atoms,
            label='crystal',
            kpts=(1,1,1)
            )

# atoms.calc = Plumed(calc=calc_base,
#                     input=setup,
#                     timestep=timestep,
#                     atoms=atoms,
#                     kT=0.1,
#                     log='../results/test/plumed/plumed_log.log')

from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin

atoms.calc = calc_base

# dyn = Langevin(atoms, 0.1, temperature_K=0.1/units.kB, friction=0.1,
#                fixcm=True, trajectory='../results/test/machine_learning/dftb_500.xyz')
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, dt=0.5*units.fs, trajectory='../results/test/machine_learning/dftb_500.traj')  
# https://wiki.fysik.dtu.dk/ase/ase/md.html

dyn.run(500)


In [ ]:
from ase.io import read, write
from ase.visualize import view

traj = read('../results/test/machine_learning/dftb_500.traj', index=":")
# traj = read('/home/dlb/Downloads/UnbiasMD.xyz', index=":")
# write("test/plumed/biased_MTD.xyz", traj, format="extxyz")
# traj_biased = read('test/plumed/biased_MTD.xyz', index=":")
# traj_unbiased = read('test/plumed/.xyz', index=":")

view(traj)
# print(traj[200].get_potential_energy())
# print(traj[200].get_forces()[33])

In [ ]:
# check if correct forces were written into traj file
# https://github.com/qsnake/ase/blob/master/ase/calculators/singlepoint.py
# atoms = read('../data/structures/new_systems/ktu_002.cif')

from ase.calculators.singlepoint import SinglePointCalculator

snap = traj[200].copy()

calc_base = Dftb(atoms=atoms,
            label='crystal',
            kpts=(1,1,1)
            )

snap.calc = calc_base

print(snap.get_potential_energy())
print(snap.get_forces()[33])

## Train GP model

In [ ]:
from ase.visualize import view
from ase import io

import numpy as np

traj_md = io.read("../results/test/machine_learning/dftb_500.traj", index=":")
energies_md = np.zeros(len(traj_md) )
forces_md = np.zeros( (len(traj_md), len(traj_md[0]), 3 ) )
for i, snap in enumerate(traj_md):
    energies_md[i] = snap.get_potential_energy()
    forces_md[i] = snap.get_forces()
# forces_md = np.load("../../data/xtb_md/forces_xtb_md.npy")
# energies_md = np.load("../../data/xtb_md/energies_xtb_md.npy")

# Training data:
# training_indices = np.sort(  np.random.choice(np.arange(200,220), 3, replace=False) )
training_indices = np.sort(  np.arange(200, 300, 5) )  
traj_train = [traj_md[i] for i in training_indices]
energies_train = energies_md[training_indices]
forces_train = forces_md[training_indices]
train_data = {'trajectory': traj_train, 'energies': energies_train, 'forces': forces_train}

#Test data:
# test_indices = np.sort(  np.random.choice(np.arange(0,92795), 200, replace=False) ) 
test_indices = np.sort(  np.arange(200,220,1) ) 
traj_test = [traj_md[i] for i in test_indices]
energies_test = energies_md[test_indices]
forces_test = forces_md[test_indices]
test_data = {'trajectory': traj_test, 'energies': energies_test, 'forces': forces_test}

data_units = "ev_angstrom" # standard ase units

In [ ]:
# using with fande environment

import sys
sys.path.append("../../fande") # Adds higher directory to python modules path.
# sys.path.append("..") # Adds higher directory to python modules path.

%load_ext autoreload
%autoreload 2

hparams = {
    'dtype' : 'float64',
    'device' : 'gpu'
}



In [ ]:

# import matplotlib.pyplot as plt
# print(fdm.forces_train_norm.shape)
# plt.plot(fdm.forces_train[:,49,0])

In [ ]:
a = np.array([1,2,3,4])

b = np.array([2,2,4,4])

a==b

In [ ]:
%%time


from fande.data import FandeDataModuleASE

# train_centers_positions = [43,49,73,79,29,35,  53,59,69,83,25,39,  75,81,31,37,45,51]
# train_derivatives_positions = [43,49,73,79,29,35,  53,59,69,83,25,39,  75,81,31,37,45,51]

train_centers_positions = list(range(264))
train_derivatives_positions = list(range(264))
# train_derivatives_positions = [25,39,53,59,69,83]
# train_centers_positions = [43
# train_derivatives_positions = [

# train_derivatives_positions = [29, 35, 43, 49, 73, 79]
# train_centers_positions = [6, 10, 14,   7, 11, 15]
# train_derivatives_positions = [28, 34, 72, 78, 42, 48,   29, 35, 43, 49, 73, 79]

test_centers_positions = [0]
test_derivatives_positions = [0]
# centers_positions = None
# derivatives_positions = None

# Descriptors parameters:
soap_params = {
    'species': ["H", "C"],
    'periodic': True,
    'rcut': 5.0,
    'sigma': 0.5,
    'nmax': 5,
    'lmax': 5,
    'average': "off",
    'crossover': True,
    'dtype': "float64",
    'n_jobs': 10,
    'sparse': False,
    'positions': [7, 11, 15] # ignored
}

fdm = FandeDataModuleASE(train_data, test_data, hparams, units=data_units)

train_gi, test_gi = fdm.calculate_invariants_librascal(
    soap_params, 
    train_centers_positions, 
    train_derivatives_positions,
    test_centers_positions, 
    test_derivatives_positions,
    same_centers_derivatives=True)


print(fdm.train_DX.shape)
print(fdm.test_DX.shape)

In [ ]:
ff = traj_md[100].get_forces().flatten()
# ff.flatten()

In [ ]:
import torch
torch.linalg.norm(fdm.train_DX[3])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(fdm.train_F[1::1], linestyle = 'None', marker='o')
plt.plot(fdm.test_F[1::1], linestyle = 'None', marker='o')

print(fdm.train_DX.shape, fdm.train_F.shape )
print(fdm.test_DX.shape, fdm.test_F.shape )

## Train Model

In [ ]:
model_f.device

In [ ]:
%time

from fande.models import ModelForces, ModelEnergies, MyCallbacks
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning import Trainer, seed_everything
import numpy as np
# seed_everything(42, workers=True)

train_DX = fdm.train_DX
train_F = fdm.train_F
test_DX = fdm.test_DX
test_F = fdm.test_F

training_random_samples = 4000#train_F.shape[0]
num_epochs = 600
learning_rate = 0.01

# ind_slice = np.sort( np.concatenate( 
#     ( np.arange(0,4800), np.arange(11*4800,12*4800), np.random.choice(np.arange(4800,59200), 300, replace=False) ) 
#     ) )

ind_slice = np.sort(  np.random.choice(np.arange(0,train_F.shape[0]), training_random_samples, replace=False) ) 
# ind_slice = np.sort(  np.arange(0,2550) ) 
# ind_slice = np.sort(  np.arange(0,train_F.shape[0]) ) 


train_dataset = TensorDataset(train_DX[ind_slice], train_F[ind_slice])
train_loader = DataLoader(train_dataset, batch_size=100_000)

model_f = ModelForces(train_DX[ind_slice], train_F[ind_slice], hparams, learning_rate)

trainer_f = Trainer(
    gpus=1, 
    max_epochs=num_epochs, 
    precision=32,
    weights_summary='full', 
    callbacks=[MyCallbacks()])

trainer_f.fit(model_f, train_loader)

In [ ]:
hypers = get_optimal_radial_basis_hypers(hypers, frames, expanded_max_radial=20)

## Predictions on test data

In [ ]:
from fande.predict import PredictorASE

model_e = None
trainer_e = None

predictor = PredictorASE(
            fdm,
            model_e,
            model_f,
            trainer_e,
            trainer_f,
            hparams,
            soap_params
)

test_centers_positions = list(range(264))
test_derivatives_positions = list(range(264))

test_gi = fdm.calculate_test_invariants_librascal(
    test_centers_positions, 
    test_derivatives_positions,
    same_centers_derivatives=True)

errors = predictor.test_errors(plot=True)


print("MSE: ", np.mean(errors**2) )
print("MAE: ", np.mean( abs(errors)) )
print("Max error: ", max(abs(errors)))

In [ ]:
print(test_gi.shape, errors.shape)

In [ ]:
## Get the indices of max erorrs

abs_errors = abs(errors)
ind_max = np.argpartition(abs_errors, -10)[-10:]
print(abs_errors[ind_max])

ind_max = ind_max%test_gi.shape[0]
print(test_gi[ind_max]%264)



In [ ]:
All 15_000 samples CG ERRORS:
MSE:  1.1444722
MAE:  0.85808694
Max error:  4.1879587

10_000 samples:
MSE:  0.075882606
MAE:  0.2096666
Max error:  2.990028

6000 samples:
MSE:  0.06504093
MAE:  0.15767942
Max error:  4.1622434

5000 samples:
MSE:  0.056790385
MAE:  0.1495894
Max error:  2.7035623

4000 samples:
MSE:  0.109407514
MAE:  0.18077059
Max error:  5.444092

3000 samples:
MSE:  0.13578652
MAE:  0.21768376
Max error:  4.5557604


2000 samples:
MSE:  0.19382845
MAE:  0.26960444
Max error:  3.7858768


1000 samples:
MSE:  0.3523444
MAE:  0.38389957
Max error:  6.225298


In [ ]:
test_centers_positions = [162]
test_derivatives_positions = [162]

test_gi = fdm.calculate_test_invariants_librascal(
    test_centers_positions, 
    test_derivatives_positions,
    same_centers_derivatives=True)

# del predictor

predictor.predict_and_plot_forces_r()

In [ ]:
from ase.visualize import view

view(traj_md)

In [ ]:

%%time
snapshot = traj_md[0]

# for i in range(5):
#     fdm.calculate_snapshot_invariants_librascal(snapshot)

# fdm.snap_DX.shape
s=0
for i in range(4):
    snapshot = traj_md[i]
    try:
        predictor.predict_forces_single_snapshot_r(snapshot)
    except:
        s = s+1
        print("\n You are a failure!!! Emotional daaaamage!!!\n")

In [ ]:
snapshot.get_pbc()

## Testing area

### MD with fande calc

In [ ]:
%%capture c
from fande.ase import FandeCalc
from runner.dynamics import MDRunner

from ase import units

import logging

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet

logging.getLogger("pytorch_lightning").setLevel(logging.ERROR) # logging.ERROR to disable or INFO

# atoms = fdm.mol_traj[10].copy()
atoms = traj_md[100].copy()
atoms.set_pbc(True)

atoms.calc = FandeCalc(predictor)

# print( atoms.get_forces() )
# print( atoms.get_potential_energy())
# "../results/test/"
# mdr = MDRunner(atoms, "../results/test/md_runs/md_test.traj", "../results/test/md_runs/md_log.log")
# mdr.run(dt=0.5*units.fs, num_steps=200)

# MaxwellBoltzmannDistribution(self.atoms, temperature_K=300)

dyn = VelocityVerlet(
    atoms,
    dt = 0.4*units.fs,
    trajectory="../results/test/md_runs/md_test.traj",
    logfile="../results/test/md_runs/md_log.log",
)

dyn.run(200)

In [ ]:
file = open("../results/test/md_runs/captured_output.log", "w")
a = file.write(c.stdout)
file.close()

In [ ]:
from ase.visualize import view
from ase.io import read

traj = read("../results/test/md_runs/md_test.traj", index=":")

view(traj)

In [ ]:
import os
import ipyparallel as ipp

cluster = ipp.Cluster(n=4)
with cluster as rc:
    ar = rc[:].apply_async(os.getpid)
    pid_map = ar.get_dict()

In [ ]:
import ipyparallel as ipp
rc = ipp.Cluster(n=4).start_and_connect_sync()

## Run simulations